Bootstrap an equivalent number of members from the 5 LE's to estimate the error on the predictor and predictand and the correlation between them for the CMIP5 models.

In [5]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import warnings

from ecpaper_utils import bootstrap_utils as boot
from ecpaper_utils import jlat_utils as jlat

importlib.reload(boot)

warnings.filterwarnings('ignore')

In [6]:
cmip5models=pd.read_csv("../../DATASORT/cmip5csvinfo.csv")

In [7]:
mmleapath="/project/cas/islas/python/ecpaper2020/DATASORT/SHJET/DATA/"

Read in MMLEA data

In [8]:
dat = xr.open_dataset(mmleapath+"uzm_CanESM2.nc")
up_CanESM2 = dat.upast_jja_zm ; uf_CanESM2 = dat.ufut_jja_zm
dat = xr.open_dataset(mmleapath+"uzm_CESM1-CAM5.nc")
up_CESM = dat.upast_jja_zm ; uf_CESM = dat.ufut_jja_zm
dat = xr.open_dataset(mmleapath+"uzm_CSIRO-Mk3-6-0.nc")
up_CSIRO = dat.upast_jja_zm ; uf_CSIRO = dat.ufut_jja_zm
dat = xr.open_dataset(mmleapath+"uzm_GFDL-CM3.nc")
up_GFDL = dat.upast_jja_zm ; uf_GFDL = dat.ufut_jja_zm
dat = xr.open_dataset(mmleapath+"uzm_MPI-ESM.nc")
up_MPI = dat.upast_jja_zm ; uf_MPI = dat.ufut_jja_zm

In [9]:
models = cmip5models['Model']
nmodels=models.size
nboots=1000

sigx = np.zeros(nmodels) ; sigy = np.zeros(nmodels) ; rxy = np.zeros(nmodels)

for index, modname in models.iteritems():
    print(modname)
    nmemsp=cmip5models.loc[index, "Nmempast"]
    nmemsf=cmip5models.loc[index, "Nmemfuture"]
    
    bootCanESM2_p = boot.bootgen_multimem(up_CanESM2, nboots, nmemsp, seed=3)
    bootCanESM2_f = boot.bootgen_multimem(uf_CanESM2, nboots, nmemsf, seed=3)
    bootCESM_p = boot.bootgen_multimem(up_CESM, nboots, nmemsp, seed=3)
    bootCESM_f = boot.bootgen_multimem(uf_CESM, nboots, nmemsf, seed=3)
    bootCSIRO_p = boot.bootgen_multimem(up_CSIRO, nboots, nmemsp, seed=3)
    bootCSIRO_f = boot.bootgen_multimem(uf_CSIRO, nboots, nmemsf, seed=3)
    bootGFDL_p = boot.bootgen_multimem(up_GFDL, nboots, nmemsp, seed=3)
    bootGFDL_f = boot.bootgen_multimem(uf_GFDL, nboots, nmemsf, seed=3)
    bootMPI_p = boot.bootgen_multimem(up_MPI, nboots, nmemsp, seed=3)
    bootMPI_f = boot.bootgen_multimem(uf_MPI, nboots, nmemsf, seed=3)
    
    jlat_CanESM2_p=np.zeros(nboots) ; jlat_CanESM2_f=np.zeros(nboots)
    jlat_CESM_p=np.zeros(nboots) ; jlat_CESM_f=np.zeros(nboots)
    jlat_CSIRO_p=np.zeros(nboots) ; jlat_CSIRO_f=np.zeros(nboots)
    jlat_GFDL_p=np.zeros(nboots) ; jlat_GFDL_f=np.zeros(nboots)
    jlat_MPI_p=np.zeros(nboots) ; jlat_MPI_f=np.zeros(nboots)
    
    for iboot in range(0,nboots):
        jlat_CanESM2_p[iboot], jspeed = jlat.calcjetlat(bootCanESM2_p[iboot, :], -80, -20)
        jlat_CanESM2_f[iboot], jspeed = jlat.calcjetlat(bootCanESM2_f[iboot, :], -80, -20)
        jlat_CESM_p[iboot], jspeed = jlat.calcjetlat(bootCESM_p[iboot, :], -80, -20)
        jlat_CESM_f[iboot], jspeed = jlat.calcjetlat(bootCESM_f[iboot, :], -80, -20)
        jlat_CSIRO_p[iboot], jspeed = jlat.calcjetlat(bootCSIRO_p[iboot, :], -80, -20)
        jlat_CSIRO_f[iboot], jspeed = jlat.calcjetlat(bootCSIRO_f[iboot, :], -80, -20)
        jlat_GFDL_p[iboot], jspeed = jlat.calcjetlat(bootGFDL_p[iboot, :], -80, -20)
        jlat_GFDL_f[iboot], jspeed = jlat.calcjetlat(bootGFDL_f[iboot, :], -80, -20)
        jlat_MPI_p[iboot], jspeed = jlat.calcjetlat(bootMPI_p[iboot, :], -80, -20)
        jlat_MPI_f[iboot], jspeed = jlat.calcjetlat(bootMPI_f[iboot, :], -80, -20)
    
    jlatdif_CanESM2 = jlat_CanESM2_f - jlat_CanESM2_p
    jlatdif_CESM = jlat_CESM_f - jlat_CESM_p
    jlatdif_CSIRO = jlat_CSIRO_f - jlat_CSIRO_p
    jlatdif_GFDL = jlat_GFDL_f - jlat_GFDL_p
    jlatdif_MPI = jlat_MPI_f - jlat_MPI_p

    result = np.corrcoef(jlat_CanESM2_p, jlatdif_CanESM2)
    rxy_CanESM2 = result[0,1]
    result = np.corrcoef(jlat_CESM_p, jlatdif_CESM)
    rxy_CESM = result[0,1]
    result = np.corrcoef(jlat_CSIRO_p, jlatdif_CSIRO)
    rxy_CSIRO = result[0,1]
    result = np.corrcoef(jlat_GFDL_p, jlatdif_GFDL)
    rxy_GFDL = result[0,1]
    result = np.corrcoef(jlat_MPI_p, jlatdif_MPI)
    rxy_MPI = result[0,1]
    
    sigx[index] = (np.std(jlat_CanESM2_p) + np.std(jlat_CESM_p) + np.std(jlat_CSIRO_p) + np.std(jlat_GFDL_p) + np.std(jlat_MPI_p))/5.
    sigy[index] = (np.std(jlatdif_CanESM2) + np.std(jlatdif_CESM) + np.std(jlatdif_CSIRO)+ np.std(jlatdif_GFDL) +  np.std(jlatdif_MPI))/5.
    rxy[index] = (rxy_CanESM2 + rxy_CESM + rxy_CSIRO + rxy_GFDL + rxy_MPI)/5.

ACCESS1-0
ACCESS1-3
bcc-csm1-1
bcc-csm1-1-m
BNU-ESM
CanESM2
CCSM4
CESM1-BGC
CESM1-CAM5
CESM1-WACCM
CMCC-CM
CMCC-CMS
CNRM-CM5
CSIRO-Mk3-6-0
EC-EARTH
FGOALS-g2
FIO-ESM
GFDL-CM3
GFDL-ESM2G
GFDL-ESM2M
GISS-E2-H
GISS-E2-R
HadGEM2-AO
HadGEM2-CC
HadGEM2-ES
inmcm4
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC5
MIROC-ESM
MIROC-ESM-CHEM
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M
NorESM1-ME


In [14]:
sigx_xr = xr.DataArray(sigx, coords=[models], dims=["Model"], name='sigx')
sigy_xr = xr.DataArray(sigy, coords=[models], dims=["Model"], name='sigy')
rxy_xr = xr.DataArray(rxy, coords=[models], dims=["Model"], name='rxy')

sigx_xr.to_netcdf("./DATA/errors_cmip5.nc")
sigy_xr.to_netcdf("./DATA/errors_cmip5.nc", mode="a")
rxy_xr.to_netcdf("./DATA/errors_cmip5.nc", mode="a")

In [12]:
print(sigx_xr)

<xarray.DataArray 'sigx' (Model: 37)>
array([0.58296299, 0.58296299, 0.58296299, 0.58296299, 0.58296299,
       0.2562448 , 0.23306926, 0.58296299, 0.33195221, 0.58296299,
       0.58296299, 0.58296299, 0.2562448 , 0.17939958, 0.58296299,
       0.58296299, 0.33195221, 0.58296299, 0.58296299, 0.58296299,
       0.40710361, 0.40710361, 0.58296299, 0.33195221, 0.33195221,
       0.58296299, 0.28604551, 0.58296299, 0.58296299, 0.33195221,
       0.58296299, 0.58296299, 0.33195221, 0.58296299, 0.58296299,
       0.58296299, 0.58296299])
Coordinates:
  * Model    (Model) object 'ACCESS1-0' 'ACCESS1-3' ... 'NorESM1-M' 'NorESM1-ME'
